In [1]:
!pip install ccxt

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.2.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.14.0-py3.12.egg is deprecated. pip 25.1 w

In [5]:
import ccxt
import pandas as pd
import time

# Инициализация биржи
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '4h'
since = exchange.parse8601('2017-08-17T00:00:00Z')  # Самая ранняя дата для BTC/USDT на Binance

# Список для хранения всех данных
all_ohlcv = []

# Текущая временная метка в миллисекундах
current_time = int(time.time() * 1000)  # Текущая дата в миллисекундах

# Ограничение на количество свечей за запрос (Binance обычно возвращает до 500 или 1000)
limit = 1000

# Загрузка данных с пагинацией
while since < current_time:
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        if not ohlcv:
            break  # Если данных больше нет, выходим
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1  # Обновляем since до следующей свечи
        print(f"Загружено {len(ohlcv)} свечей до {pd.to_datetime(since, unit='ms')}")
        time.sleep(1)  # Задержка, чтобы не превысить лимит запросов API
    except Exception as e:
        print(f"Ошибка: {e}")
        break

# Преобразование в DataFrame
df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Удаление дубликатов, если есть
df = df[~df.index.duplicated(keep='first')]

# Сохранение в CSV
df.to_csv('btc_usdt_4h_full.csv')
print(f"Данные сохранены в 'btc_usdt_4h_full.csv'. Всего строк: {len(df)}")

Загружено 1000 свечей до 2018-01-30 16:00:00.001000
Загружено 1000 свечей до 2018-07-18 00:00:00.001000
Загружено 1000 свечей до 2018-12-31 20:00:00.001000
Загружено 1000 свечей до 2019-06-17 00:00:00.001000
Загружено 1000 свечей до 2019-11-30 20:00:00.001000
Загружено 1000 свечей до 2020-05-15 16:00:00.001000
Загружено 1000 свечей до 2020-10-29 08:00:00.001000
Загружено 1000 свечей до 2021-04-14 00:00:00.001000
Загружено 1000 свечей до 2021-09-27 16:00:00.001000
Загружено 1000 свечей до 2022-03-13 08:00:00.001000
Загружено 1000 свечей до 2022-08-27 00:00:00.001000
Загружено 1000 свечей до 2023-02-09 16:00:00.001000
Загружено 1000 свечей до 2023-07-26 08:00:00.001000
Загружено 1000 свечей до 2024-01-09 00:00:00.001000
Загружено 1000 свечей до 2024-06-23 16:00:00.001000
Загружено 1000 свечей до 2024-12-07 08:00:00.001000
Загружено 685 свечей до 2025-03-31 12:00:00.001000
Данные сохранены в 'btc_usdt_4h_full.csv'. Всего строк: 16685


In [7]:
!pip install pandas_ta

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.2.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.14.0-py3.12.egg is deprecated. pip 25.1 w

In [11]:
import pandas as pd
import pandas_ta as ta

# Загрузка данных
df = pd.read_csv('btc_usdt_4h_full.csv', index_col='timestamp', parse_dates=True)

# Differencing
for k in range(1, 6):
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[f'{col}_diff_{k}'] = df[col] / df[col].shift(k)

# EBSW
ebsw_result = ta.ebsw(df['close'])
df['ebsw'] = ebsw_result

# Chaikin Money Flow
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)

# VWAP
df['tp'] = (df['low'] + df['high'] + df['close']) / 3
df['vwap'] = (df['tp'] * df['volume']).cumsum() / df['volume'].cumsum()
df['vwap_open'] = df['vwap'] / df['open']

# Other Ratios
df['high_low'] = df['high'] / df['low']
df['high_open'] = df['high'] / df['open']
df['close_open'] = df['close'] / df['open']
df['low_open'] = df['low'] / df['open']

# Time Information
df['hour'] = df.index.hour
df['day'] = df.index.day
df['month'] = df.index.month

# Удаляем строки с NaN
df.dropna(inplace=True)

df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']

def classify(u, v):
    if u >= 0.0075:
        return 0  # c0
    elif v <= -0.0075:
        return 1  # c1
    else:
        return 2  # c2

df['target'] = [classify(u, v) for u, v in zip(df['u_t1'], df['v_t1'])]
df.dropna(inplace=True)  # Удаляем последнюю строку

# Сохранение обработанных данных
df.to_csv('btc_usdt_4h_with_target.csv')

/tmp/ipykernel_157/429230575.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ebsw_result = ta.ebsw(df['close'])


In [19]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

# Разделение данных (оригинальное из статьи)
train_df = df['2017-08-23 16:00:00':'2022-01-27 16:00:00']
val_df = df['2022-01-28 00:00:00':'2022-02-13 20:00:00']
test_df = df['2022-02-14 00:00:00':'2022-04-23 00:00:00']

# train_df = df['2017-08-23 16:00:00':'2024-06-28 16:00:00']
# val_df = df['2024-06-29 00:00:00':'2024-08-22 16:00:00']
# test_df = df['2024-08-23 00:00:00':'2025-03-31 08:00:00']

# Признаки и целевая переменная
features = [col for col in df.columns if col not in ['high_next', 'low_next', 'u_t1', 'v_t1', 'target']]
X_train = train_df[features].values
y_train = train_df['target'].values
X_val = val_df[features].values
y_val = val_df['target'].values
X_test = test_df[features].values
y_test = test_df['target'].values

# Проверка на NaN и inf
print("Проверка X_train:")
print("NaN:", np.isnan(X_train).sum())
print("Inf:", np.isinf(X_train).sum())
print("Максимальное значение:", np.max(np.abs(X_train)))

print("Проверка X_val:")
print("NaN:", np.isnan(X_val).sum())
print("Inf:", np.isinf(X_val).sum())
print("Максимальное значение:", np.max(np.abs(X_val)))

print("Проверка X_test:")
print("NaN:", np.isnan(X_test).sum())
print("Inf:", np.isinf(X_test).sum())
print("Максимальное значение:", np.max(np.abs(X_test)))

# Обработка inf и слишком больших значений
X_train = np.where(np.isinf(X_train), np.nan, X_train)  # Заменяем inf на NaN
X_train = np.nan_to_num(X_train, nan=0.0, posinf=1e10, neginf=-1e10)  # Ограничиваем значения
X_val = np.where(np.isinf(X_val), np.nan, X_val)
X_val = np.nan_to_num(X_val, nan=0.0, posinf=1e10, neginf=-1e10)
X_test = np.where(np.isinf(X_test), np.nan, X_test)
X_test = np.nan_to_num(X_test, nan=0.0, posinf=1e10, neginf=-1e10)

# Повторная проверка
print("\nПосле обработки:")
print("X_train Inf:", np.isinf(X_train).sum())
print("X_val Inf:", np.isinf(X_val).sum())
print("X_test Inf:", np.isinf(X_test).sum())

print("Распределение y_train:", np.bincount(y_train))
print("Распределение y_val:", np.bincount(y_val))
print("Распределение y_test:", np.bincount(y_test))

# class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weights_dict = dict(enumerate(class_weights))
# print("Веса классов:", class_weights_dict)

class_weights_dict = {0: 1.0, 1: 1.5, 2: 2.0}
print("Новые веса классов:", class_weights_dict)

# Инициализация и обучение TabNet
clf = TabNetClassifier(
    n_d=32, n_a=32, n_steps=5, gamma=1.3,
    lambda_sparse=0.01,  # Увеличиваем регуляризацию
    optimizer_params=dict(lr=5e-3),  # Уменьшаем learning rate
    verbose=1
)
clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy'],
    max_epochs=300,
    patience=30,
    batch_size=256,
    virtual_batch_size=128,
    weights=class_weights_dict
)

# Предсказание и оценка
y_pred = clf.predict(X_test)

# Вывод распределения предсказаний и истинных меток
print("Распределение предсказанных классов:", np.bincount(y_pred))
print("Распределение истинных классов:", np.bincount(y_test))

# Метрики
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")

# Подробный отчёт
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['c0', 'c1', 'c2']))

Проверка X_train:
NaN: 0
Inf: 5
Максимальное значение: inf
Проверка X_val:
NaN: 0
Inf: 0
Максимальное значение: 45821.0
Проверка X_test:
NaN: 0
Inf: 0
Максимальное значение: 48189.84

После обработки:
X_train Inf: 0
X_val Inf: 0
X_test Inf: 0
Распределение y_train: [4834 2867 1992]
Распределение y_val: [43 41 18]
Распределение y_test: [183 120 106]
Новые веса классов: {0: 1.0, 1: 1.5, 2: 2.0}


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.37539 | val_0_accuracy: 0.46078 |  0:00:01s
epoch 1  | loss: 1.09019 | val_0_accuracy: 0.48039 |  0:00:02s
epoch 2  | loss: 1.05291 | val_0_accuracy: 0.41176 |  0:00:04s
epoch 3  | loss: 1.01492 | val_0_accuracy: 0.43137 |  0:00:05s
epoch 4  | loss: 1.01511 | val_0_accuracy: 0.41176 |  0:00:07s
epoch 5  | loss: 0.99945 | val_0_accuracy: 0.34314 |  0:00:08s
epoch 6  | loss: 0.993   | val_0_accuracy: 0.39216 |  0:00:10s
epoch 7  | loss: 0.98615 | val_0_accuracy: 0.39216 |  0:00:11s
epoch 8  | loss: 0.96213 | val_0_accuracy: 0.40196 |  0:00:12s
epoch 9  | loss: 0.96254 | val_0_accuracy: 0.42157 |  0:00:14s
epoch 10 | loss: 0.96466 | val_0_accuracy: 0.42157 |  0:00:15s
epoch 11 | loss: 0.96392 | val_0_accuracy: 0.44118 |  0:00:17s
epoch 12 | loss: 0.9582  | val_0_accuracy: 0.42157 |  0:00:18s
epoch 13 | loss: 0.95292 | val_0_accuracy: 0.43137 |  0:00:19s
epoch 14 | loss: 0.95116 | val_0_accuracy: 0.41176 |  0:00:21s
epoch 15 | loss: 0.94921 | val_0_accuracy: 0.41176 |  0

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Распределение предсказанных классов: [184 210  15]
Распределение истинных классов: [183 120 106]
Accuracy: 0.3570
F1-score: 0.3351

Classification Report:
               precision    recall  f1-score   support

          c0       0.46      0.46      0.46       183
          c1       0.26      0.45      0.33       120
          c2       0.53      0.08      0.13       106

    accuracy                           0.36       409
   macro avg       0.42      0.33      0.31       409
weighted avg       0.42      0.36      0.34       409

